In this toturial, we want to integrate Keras models with [Tensorflow Estimators](https://www.tensorflow.org/extend/estimators). 

This tutorial essentially follow the same approach described in [this tutorial](https://www.tensorflow.org/extend/estimators), however, instead of using Tensorflow layers, we want to use Keras layer.

Our model consists of two fully connected layers with the relu activation function followed by a linear layer without non-linearity. So, lets implement it with Kera:

In [3]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten

# # This returns a tensor
# inputs = Input(shape=(784,))

# # a layer instance is callable on a tensor, and returns a tensor
# x = Dense(64, activation='relu')(inputs)
# x = Dense(64, activation='relu')(x)
# predictions = Dense(10, activation='softmax')(x)



In [12]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""DNNRegressor with custom estimator for abalone dataset."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

from six.moves import urllib

import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from tensorflow.contrib.learn.python.learn.estimators.estimator import SKCompat as SKCompat
from tensorflow.contrib.learn import Estimator as Estimator

FLAGS = None

tf.logging.set_verbosity(tf.logging.INFO)

# Learning rate for the model
LEARNING_RATE = 0.001


def maybe_download(train_data, test_data, predict_data):
  """Maybe downloads training data and returns train and test file names."""
  if train_data:
    train_file_name = train_data
  else:
    train_file = tempfile.NamedTemporaryFile(delete=False)
    urllib.request.urlretrieve(
        "http://download.tensorflow.org/data/abalone_train.csv",
        train_file.name)
    train_file_name = train_file.name
    train_file.close()
    print("Training data is downloaded to %s" % train_file_name)

  if test_data:
    test_file_name = test_data
  else:
    test_file = tempfile.NamedTemporaryFile(delete=False)
    urllib.request.urlretrieve(
        "http://download.tensorflow.org/data/abalone_test.csv", test_file.name)
    test_file_name = test_file.name
    test_file.close()
    print("Test data is downloaded to %s" % test_file_name)

  if predict_data:
    predict_file_name = predict_data
  else:
    predict_file = tempfile.NamedTemporaryFile(delete=False)
    urllib.request.urlretrieve(
        "http://download.tensorflow.org/data/abalone_predict.csv",
        predict_file.name)
    predict_file_name = predict_file.name
    predict_file.close()
    print("Prediction data is downloaded to %s" % predict_file_name)

  return train_file_name, test_file_name, predict_file_name


def model_fn(features, targets, mode, params):
  """Model function for Estimator."""
  print("test")
  # Connect the first hidden layer to input layer
  # (features) with relu activation
  first_hidden_layer = tf.contrib.layers.relu(features, 10)

  # Connect the second hidden layer to first hidden layer with relu
  second_hidden_layer = tf.contrib.layers.relu(first_hidden_layer, 10)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.contrib.layers.linear(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"ages": predictions}

  # Calculate loss using mean squared error
  loss = tf.losses.mean_squared_error(targets, predictions)

  # Calculate root mean squared error as additional eval metric
  eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(
          tf.cast(targets, tf.float32), predictions)
  }

  train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=params["learning_rate"],
      optimizer="SGD")

  return model_fn_lib.ModelFnOps(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops)



parser = argparse.ArgumentParser()
parser.register("type", "bool", lambda v: v.lower() == "true")
parser.add_argument(
  "--train_data", type=str, default="", help="Path to the training data.")
parser.add_argument(
  "--test_data", type=str, default="", help="Path to the test data.")
parser.add_argument(
  "--predict_data",
  type=str,
  default="",
  help="Path to the prediction data.")
FLAGS, unparsed = parser.parse_known_args()


# def main(unused_argv):
  # Load datasets
abalone_train, abalone_test, abalone_predict = maybe_download(
    FLAGS.train_data, FLAGS.test_data, FLAGS.predict_data)

# Training examples
training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=abalone_train, target_dtype=np.int, features_dtype=np.float32)

# Test examples
test_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=abalone_test, target_dtype=np.int, features_dtype=np.float32)

# Set of 7 examples for which to predict abalone ages
prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=abalone_predict, target_dtype=np.int, features_dtype=np.float32)

# Set model params
model_params = {"learning_rate": LEARNING_RATE}

# Instantiate Estimator
est = Estimator(model_fn=model_fn, params=model_params)
nn = SKCompat(est)

# Fit
nn.fit(x=training_set.data, y=training_set.target, steps=5000)

# Score accuracy
ev = est.evaluate(x=test_set.data, y=test_set.target, steps=1)
print("Loss: %s" % ev["loss"])
print("Root Mean Squared Error: %s" % ev["rmse"])

# Print out predictions
predictions = nn.predict(x=prediction_set.data)
print(predictions)
for i, p in enumerate(predictions['ages']):
  print("Prediction %s: %s" % (i + 1, p))


# tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Training data is downloaded to /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmpqzf0g8aj
Test data is downloaded to /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmpapbmua_m
Prediction data is downloaded to /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmp8gysvgjq
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_model_dir': None, '_num_ps_replicas': 0, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_master': '', '_save_checkpoints_secs': 600, '_task_id': 0, '_task_type': None, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11726e7b8>, '_num_worker_replicas': 0, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
}
test
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/_g/9zy1zpy55c

/Users/majid/anaconda/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Starting evaluation at 2017-04-05-23:55:02
INFO:tensorflow:Restoring parameters from /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmptfjinenw/model.ckpt-5000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-04-05-23:55:02
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 5.53387, rmse = 2.35242
Loss: 5.53387
Root Mean Squared Error: 2.35242
test
INFO:tensorflow:Restoring parameters from /var/folders/_g/9zy1zpy55c59kbl62wkkj6hm0000gn/T/tmptfjinenw/model.ckpt-5000
{'ages': array([  4.6201067 ,  10.41481686,   7.14297915,  10.65043831,
        10.94178391,   9.2508297 ,  11.34892464], dtype=float32)}
Prediction 1: 4.62011
Prediction 2: 10.4148
Prediction 3: 7.14298
Prediction 4: 10.6504
Prediction 5: 10.9418
Prediction 6: 9.25083
Prediction 7: 11.3489


In [8]:
predictions['ages']

array([  4.57161617,  10.36000156,   7.23409271,  10.63977051,
        11.09867287,   9.45869637,  11.08108139], dtype=float32)

In [10]:
for i, p in enumerate(predictions['ages']):
  print("Prediction %s: %s" % (i + 1, p))


Prediction 1: 4.57162
Prediction 2: 10.36
Prediction 3: 7.23409
Prediction 4: 10.6398
Prediction 5: 11.0987
Prediction 6: 9.4587
Prediction 7: 11.0811
